# COGS 108 - Final Project

## Important

- Make sure all group member (3-5 people) are listed in the group members section.
- ONE, and only one, member of your group should upload this notebook to TritonED. 
- Each member of the group will receive the same grade on this assignment. 
- Keep the file name the same: submit the file 'A4_ProjectProposal.ipynb'

## Overview

Read the project description, and detailed instructions for this assignment, in the 'A4_ProjectOutlineAndProposal' pdf.

## Group Members: Fill in the Student ID's of each group member here
- A13353652 
- A13749314
- A13952215

### Research Question

Which counties of the United States are most in need for hospitals? 

### Hypothesis

Rural areas of the United States will need more hospitals than urban areas.
The reason being that the population in urban cities are higher.

### Dataset(s)

- Dataset Name: Hospitals
- Link to the dataset: https://hifld-dhs-gii.opendata.arcgis.com/datasets/e13641c764344b8ab7dfd41831e56940_0
- Number of observations: 7332

1-2 sentences describing the dataset. 
    This database contains locations of Hospitals for 50 US states.
    The database does not contain nursing homes or health centers.

- Dataset Name: 2010 Census Population Estimation
- Link to the dataset: https://www2.census.gov/programs-surveys/popest/datasets/2010/2010-eval-estimates/
- Number of observations: 3195

1-2 sentences describing the dataset. 
    This dataset contains the 2010 population estimate by county.

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.
    Sort the hospital dataset by county, then add the number of hospitals in that county.
    Then, we combine the census dataset with the hospital set by their counties.

### Background & Prior Work

Referencess (include a link):
- 1) This article talks about how poor neighborhoods in the U.S. don't have enough access to medical care.
    http://newsinteractive.post-gazette.com/longform/stories/poorhealth/1/
- 2) This article talks about why there are less doctors in the rural area.
    https://www.theatlantic.com/health/archive/2014/08/why-wont-doctors-move-to-rural-america/379291/

### Proposed Methods

*Fill in your proposed methods information here:*

*Data Cleaning (2-3 sentences)*
    The census and the hospital data seem to have no holes in their data set. The only cleaning we need to do to merge the two data sets together properly is to remove the word, “county” in the census data for every row in the county column.
    
*Data Analysis (1-2 paragraphs)*
    We will be able to do a data analysis by an r correlation coefficient. Visual data analysis by having scatter plot with an x axis of population in a given county and y axis having the number of hospitals in that county. From this we can calculate a regression line of best fit as well as see a trend of the hospital to county. 

*What to report (2-3 sentences)*
    We will report if there is a trend between population and the number of hospitals in a particular county. From this, we can also make recommendation to counties that have too few hospitals in their area. 

Packages we plan to use:
- a. panda
- b. matplotlib.pyplot
- c. numpy

In [22]:
# Imports
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

## Part 1: Load & Clean the Data

In [26]:
hospitals = pd.read_csv('Hospitals.csv')
census = pd.read_csv('census.csv')
#census
hospitals


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte

### Discussion

The two sets that we have received are from different years. The data we received from the population is from the United States census which is older than the data we have for the hospitals. The data on the hospitals were received from the various state departments and federal resources which may not be missing some hospitals. 

Some counties may have other hospitals close by in a neighboring county if there are lower populations in the county. Another bias would be urban areas vs. rural area since there is a bigger population of people living in urban areas than rural. Also the number of hospitals does not represent the number of people using the hospitals. The size of the hospital are also not taken into account from our data of the hospitals because less amount of hospitals does not mean that less people are being served.

If our methods do not work or our hypotheses are wrong, then we will reflect on our mistakes and change the category of the data that we are looking at. Like for example instead of the ratio between the number of hospitals and the population of the county, we could change the two variables that we are comparing that receive a more accurate conclusion.